In [ ]:
import matplotlib.pyplot as plt
import main as m

In [ ]:
model = m.Model()


In [ ]:
# plot the grid
plt.imshow(model.grid, cmap='Set1')
plt.clim(0,10)
plt.show()

In [ ]:
model.loop(1000)

In [ ]:
plt.imshow(model.grid, cmap='Set1')
plt.title("Final grid 2")
plt.show()

In [ ]:
print(model.get_fractal_dim(10))